# Example: Pricing American Options in rough Bergomi

In this notebook we present how to use the code from https://github.com/lucapelizzari/Optimal_Stopping_with_signatures/tree/main to compute lower and upper bounds for American options in the rough Bergomi model using the Signature methods, see Section 4.2 of https://arxiv.org/abs/2312.03444. Recall the price and volatility dynamics of the latter are given by \begin{align*}\label{eq:PriceDynamics}
dX_t &= rX_tdt+X_tv_t \left (\rho dW_r+\sqrt{1-\rho^2}dB_t\right ), \\ v_t & =\xi_0\mathcal{E}\left (\eta \int_0^t(t-s)^{H-\frac{1}{2}}dW_s \right )
\end{align*} and pricing an American Put-option can be formulated at optimal stopping problem $$y_0=\sup_{\tau \in \mathcal{S}_0}\mathbb{E}[e^{-r\tau}\left (K-X_{\tau}\right )^{+}]$$ for some strike $K$. In these experiments we consider the choice $$ H=0.07,X_0 = 100, r=0.05, \eta = 1.9, \rho = -0.9, \xi_0= 0.09, K = 110$$.


## Step 1: Import packages 

The iisignature package is used to compute the path-signature, and the gurobi package will be applied to solve efficiently solve high-dimensional linear programs. A license from the latter is required, and one can obtain a free-license on https://www.gurobi.com/academia/academic-program-and-licenses/.

In [1]:
!pip install iisignature
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 11.0 MB/s eta 0:00:0000:0100:01


## Step 2: Define parameters for the optimal stopping problem

In [2]:
import numpy as np
H = 0.07 
eta = 1.9
X0 = 100
r = 0.05
rho = -0.9
xi = 0.09
strike = 110
def phi(x):
    return np.maximum(strike-x,0)
phi = [phi]

Next we fix the parameters for the primal and dual procedure (sample size,exercise and discretization grid, signature truncation, degree of state-polynomials)

In [1]:
K_primal = 4 #Signature trunctation primal
K_dual = 3 #Signature truncation dual
KK_primal = 2 #polynomial levels of states
KK_dual = 2 #polynomial levels of states
N = 12 #number of stopping dates
J = 120 #discretization-grid for computing the siganture (that is iterated integrals)
T = 1 #Final time
M1_primal = 80000 #number of samples for the Longstaff-Schwartz procedure
M2_primal = 80000 #number of samples for independent resimulation to get lower-bounds
M1_dual = 4000 #number of samples for the SAA dual procedure
M2_dual = 20000 #number of samples for independent resimulation to get lower-bounds

The file American_Option_Pricing_rBergomi.py contains the functions to compute lower and upper-bounds

In [3]:
from American_Option_Pricing_rBergomi import LongstaffSchwartz_signature_rBergomi, DualSAA_signature_rBergomi

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Step 3: Compute lower bounds

The function LongstaffSchwartz_signature_rBergomi computes (true) lower bounds, by first solving the Longstaff-Schwartz algorithm with signatures + state-polynomials described in Section 4.2 of the paper, using M1_primal samples of rough Bergomi prices. This returns linear functionals of the truncated signature at each exercise date (approxmation for continuation values). By independently resimulating M2_primal paths and their signature and polynomials, the function uses the stopping policies for each path to decide the optimal stopping point, and then averaging over all samples gives a true lower bounds for the Option price.

In [4]:
y_lowerbound,stopping_policies,run_time_primal = LongstaffSchwartz_signature_rBergomi(M1_primal,M2_primal,J,N,T,phi,rho,K_primal,KK_primal,X0,H,xi,eta,r)

In [6]:
print('a lower bound is given by',y_lowerbound)

a lower bound is given by [13.23074721]


## Step 4: Compute upper bounds

The function DualSAA_signature_rBergomi computes (true) upper bounds, by minimizing over the familiy of signature-martingales, by solving the linear programm described in Remark 3.10 and Section 4.2 in the paper, along the M1_dual sample-paths, which returns optimal coefficients, and hence a martingale by considering the linear combination of the family of martingales with respect to these coefficients (approximation of Doob-martingale). After independently resimulating M2_dual paths, the function computes the Doob-Martingales for the given coefficients for all samples, and get a true upper-bound for the price by averaging.

Attention: To run the following you need to have a license from Gurobi, here is where you can get it and how to install it https://www.gurobi.com/academia/academic-program-and-licenses/. Also notice that depending on the number of samples, it can take some time to run the code (e.g. for M1_dual = 5000 and K_dual=4 and KK_dual = 3, 1-2 minutes).

In [ ]:
y_upperbound,coefficients,run_time_dual = DualSAA_signature_rBergomi(M1_dual,M2_primal,J,N,T,phi,rho,K_dual,KK_dual,X0,H,xi,eta,r)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
8.830441951751709


Finally, we can print the interval of the lower and upper bounds, and it can be compared with the bounds computed in Section 4.2 of the paper.

In [ ]:
print('The true values lies within in interval',[y_lowerbound,y_upperbounds])